This notebook contains code that performs a dynamic sentiment analysis based on emojis.

Emojis are extracted from tweets, and the tweet text is used to predict the emoji.

In [1]:
from extract_tweets import get_raw_tweets
import pickle

raw_tweets = get_raw_tweets('BackLAOut',5e6)
raw_tweets += get_raw_tweets('BackNYOut',5e6)

with open('raw_BackLA_BackNY', 'wb') as fp:
    pickle.dump(raw_tweets, fp)

500000 of 5000000.0 tweets read
1000000 of 5000000.0 tweets read
1500000 of 5000000.0 tweets read
2000000 of 5000000.0 tweets read
2500000 of 5000000.0 tweets read
3000000 of 5000000.0 tweets read
3500000 of 5000000.0 tweets read
4000000 of 5000000.0 tweets read
4500000 of 5000000.0 tweets read
500000 of 5000000.0 tweets read
1000000 of 5000000.0 tweets read
1500000 of 5000000.0 tweets read
2000000 of 5000000.0 tweets read
2500000 of 5000000.0 tweets read
3000000 of 5000000.0 tweets read
3500000 of 5000000.0 tweets read
4000000 of 5000000.0 tweets read
4500000 of 5000000.0 tweets read


In [2]:
from extract_tweets import clean_tweets
import pickle
with open('raw_BackLA_BackNY', 'rb') as fp:
    raw_tweets = [tweet for tweet in pickle.load(fp)]
    
cleaned_tweets = clean_tweets(raw_tweets)

with open('clean_BackLA_BackNY', 'wb') as fp:
    pickle.dump(cleaned_tweets, fp)
    
len(cleaned_tweets)

10.0% read
20.0% read
30.0% read
40.0% read
50.0% read
60.0% read
70.0% read
80.0% read
90.0% read
100.0% read


1045696

In [3]:
import random
import emoji
from itertools import chain
from collections import OrderedDict
from extract_tweets import get__X_and__y
import pickle
with open('clean_BackLA_BackNY', 'rb') as fp:
    cleaned_tweets = [tweet for tweet in pickle.load(fp)]

random.shuffle(cleaned_tweets)

emojis = emoji.core.unicode_codes.UNICODE_EMOJI
emoji_sentiment1 = dict(zip(emojis, [0]*len(emojis)))
emoji_sentiment2 = \
OrderedDict([('☹', -1),('👺', -1),('👿', -1),('💔', -1),('💙', -1),('🖤', -1),('😑', -1),
             ('😒', -1),('😓', -1),('😔', -1),('😕', -1),('😖', -1),('😞', -1),('😟', -1),
             ('😠', -1),('😡', -1),('😢', -1),('😣', -1),('😤', -1),('😥', -1),('😦', -1),
             ('😧', -1),('😨', -1),('😩', -1),('😫', -1),('😭', -1),('😰', -1),('😱', -1),
             ('😲', -1),('😳', -1),('😵', -1),('😾', -1),('😿', -1),('🙁', -1),('🙄', -1),
             ('🤒', -1),('🤕', -1),('🤢', -1),('🤥', -1),('🤬', -1),('🤮', -1),('🥵', -1),
             ('🥺', -1),('🤦', -1),('🤦🏻', -1),('🤦🏼', -1),('🤦🏽', -1),('🤦🏾', -1),('🤦🏿', -1),
             
             ('☺',  1),('♥',  1),('❣',  1),('❤',  1),('🐱',  1),('👅',  1),('💑',  1),
             ('💓',  1),('💕',  1),('💖',  1),('💗',  1),('💘',  1),('💚',  1),('💛',  1),
             ('💜',  1),('💝',  1),('💞',  1),('💟',  1),('😀',  1),('😁',  1),('😂',  1),
             ('😃',  1),('😄',  1),('😅',  1),('😆',  1),('😇',  1),('😈',  1),('😉',  1),
             ('😊',  1),('😋',  1),('😌',  1),('😍',  1),('😎',  1),('😏',  1),('😗',  1),
             ('😘',  1),('😙',  1),('😚',  1),('😛',  1),('😜',  1),('😝',  1),('😸',  1),
             ('😺',  1),('😻',  1),('😼',  1),('😽',  1),('🙂',  1),('🙃',  1),('🤍',  1),
             ('🤎',  1),('🤑',  1),('🤗',  1),('🤛',  1),('🤛🏻',  1),('🤛🏼',  1),('🤛🏽',  1),
             ('🤛🏾',  1),('🤛🏿',  1),('🤜',  1),('🤜🏻',  1),('🤜🏼',  1),('🤜🏽',  1),('🤜🏾',  1),
             ('🤜🏿',  1),('🤠',  1),('🤡',  1),('🤤',  1),('🤪',  1),('🤭',  1),('🥰',  1),
             ('🥳',  1),('🥴',  1),('🧡',  1)])
for e in emojis:
    if e in emoji_sentiment2:
        del emoji_sentiment1[e]
emoji_sentiment = OrderedDict(chain(emoji_sentiment1.items(),emoji_sentiment2.items()))

with open('emoji_sentiment', 'wb') as fp:
    pickle.dump(emoji_sentiment, fp)

In [4]:
from extract_tweets import get__X_and__y
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

_X, _y = get__X_and__y(cleaned_tweets, emoji_sentiment)

with open('_X', 'wb') as fp:
    pickle.dump(_X, fp)
with open('_y', 'wb') as fp:
    pickle.dump(_y, fp)

In [12]:
from collections import Counter, OrderedDict
from math import ceil
import numpy as np
from functools import reduce
import pickle
with open('_X', 'rb') as fp:
    _X = [tweet for tweet in pickle.load(fp)]
with open('_y', 'rb') as fp:
    _y = pickle.load(fp)

batch_size = 64

def split_tr_val_te(X):
    percent_tr = 1/3
    percent_val = 1/3

    tr_size = batch_size*ceil(percent_tr*len(X)/batch_size)
    val_size = ceil(percent_val*(len(X)-tr_size)/(1-percent_tr))
    te_size = len(X) - tr_size - val_size

    X_val = X[:val_size]
    X_tr = X[val_size:-te_size]
    X_te = X[-te_size:]
    print(len(X),len(X_tr)/len(X),len(X_val)/len(X),len(X_te)/len(X),len(X_tr)/batch_size)
    return X_tr, X_val, X_te

_X_tr,_X_val,_X_te = split_tr_val_te(_X)
_y_tr,_y_val,_y_te = split_tr_val_te(_y)

def split_tweets_into_words(_X):
    return [[word for word in tweet.split(' ')] for tweet in _X]

_X_tr = split_tweets_into_words(_X_tr)
_X_val = split_tweets_into_words(_X_val)
_X_te = split_tweets_into_words(_X_te)

emoji_counts = Counter()
for i in _y_tr:
    emoji_counts[i] += 1
emoji_counts, {k:v/sum(emoji_counts.values()) for k,v in emoji_counts.items()}

995418 0.3333674898384397 0.3333162550807801 0.3333162550807801 5185.0
995418 0.3333674898384397 0.3333162550807801 0.3333162550807801 5185.0


(Counter({1: 132607, 0: 101807, -1: 97426}),
 {1: 0.3996112584378014, 0: 0.30679544358727096, -1: 0.29359329797492767})

In [17]:
# get word counts and vocabulary size
runonsentence = ' '.join([' '.join(tweet) for tweet in _X_tr]+
                         [' '.join(tweet) for tweet in _X_val]+
                         [' '.join(tweet) for tweet in _X_te]).split(' ')

words = np.unique(runonsentence)
def get_counts_adv(_X):
    d1 = dict(zip(words,[0]*len(words)))
    d2 = dict(zip(words,[0]*len(words)))
    d3 = dict(zip(words,[0]*len(words)))
    
    for x in _X:
        for w in x:
            d1[w] += 1
    m1 = np.mean(list(d1.values()))
    sd1 = np.std(list(d1.values()))
    print(m1,sd1)
    
    for x in _X:
        d = Counter()
        for w in x:
            d[w] = 1
        for w in d:
            d2[w] += d[w]
    m2 = np.mean(list(d2.values()))
    sd2 = np.std(list(d2.values()))
    print(m2,sd2)
    
    for w in set(w for x in _X for w in x):
        d3[w] = (d1[w] - m1) / sd1 + (d2[w] - m2) / sd2

    return d3

wc_tr = get_counts_adv(_X_tr)
wc_val = get_counts_adv(_X_val)
wc_te = get_counts_adv(_X_te)

for w in words:
    if wc_tr[w]*wc_val[w]*wc_te[w]==0:
        del wc_tr[w]
        del wc_val[w]
        del wc_te[w]
        
wc_tr

7.207166258222624 301.2477423951703
6.839419579517606 257.5334969141633
7.209709789758803 300.77187860744795
6.84275506255643 257.55213194739883
7.209787179156455 300.2298956841641
6.843743067199793 257.2424059945996


{'': 39.520146497064005,
 '0': 1.2077048382160902,
 '00': 0.08636604193081852,
 '000': 0.014340872149753357,
 '0000': 0.03594842308407291,
 '0001': -0.021671712740779232,
 '001': 0.03594842308407291,
 '002': -0.04327926367509878,
 '003': -0.02887422971888575,
 '004': -0.04327926367509878,
 '006': -0.03607674669699227,
 '007': 0.06475849099649897,
 '009': -0.02887422971888575,
 '01': 0.2088088305586293,
 '010': -0.014469195762672714,
 '0100': -0.021671712740779232,
 '0101': -0.02887422971888575,
 '0108': -0.02887422971888575,
 '011': -0.04327926367509878,
 '0110': -0.03607674669699227,
 '012': -0.04327926367509878,
 '0123': -0.03607674669699227,
 '013': -0.03607674669699227,
 '016': -0.02887422971888575,
 '019': -0.03607674669699227,
 '02': 0.17279624566809673,
 '021': -0.021671712740779232,
 '023': -0.021671712740779232,
 '024': -0.03607674669699227,
 '025': -0.03607674669699227,
 '03': 0.1511886947337772,
 '030': -0.04327926367509878,
 '0309958207gtbank': -0.02887422971888575,
 '0312'

In [18]:
def get_most_frequent_keys(d,n):
    a = np.array(list(d.values()),dtype='float')
    a -= np.nanmin(a)
    N = min(n,len(d))
    inds = a >= 0.99 * min(a[np.argpartition(a,-N)][-N:])
    d = dict(np.array(list(d.items()))[inds])
    for k in d:
        d[k] = float(d[k])
    odct = OrderedDict(sorted(d.items(),key=lambda x:x[1], reverse=True))
    return odct

vocab = get_most_frequent_keys(wc_tr,5000)
print(len(vocab))

def get_counts(i):
    d = Counter()
    for j in i:
        d[j] += 1
    return d

runonsentence = [w if w in vocab else '<UNK>' for w in runonsentence]
word_counts = get_counts(runonsentence)
    
assert(min(list(word_counts.values())) > 1)

word_counts = OrderedDict(sorted(word_counts.items()), key=lambda x: x[0], reverse=True)

vocabulary_size = len(word_counts)
vocabulary_size, word_counts['<UNK>'], _X_tr[:10]

5040


(5042,
 1280279,
 [['i', '️', 'donnie', 'excellent', 'thematic', 'consistency'],
  ['tanning',
   'for',
   'the',
   'first',
   'time',
   'in',
   'months',
   'i',
   'love',
   'you',
   'sun',
   '️'],
  ['this', 'new', 'weeknd', 'album', 'is', 'pretty', 'good', 'so', 'far'],
  ['factsssss'],
  ['quiero', 'un', 'activo', 'asi'],
  ['oh', 'man', 'this', 'made', 'my', 'night'],
  ['better', 'days', 'ahead'],
  ['kannnnn', 'craving', 'lakkk'],
  ['aht', 'aht', 'i', 'haven’t', 'met', 'one'],
  ['still',
   'going',
   'to',
   'work',
   'cus',
   'somehow',
   'people',
   'in',
   'la',
   'still',
   'want',
   'their',
   'dogs',
   'groomed',
   'with',
   'all',
   'this',
   'rain',
   'coronavirus',
   'out',
   'there']])

In [19]:
T=np.array([(0,len(x)) for x in _X_tr]+
           [(1,len(x)) for x in _X_val]+
           [(2,len(x)) for x in _X_te])
T=np.vstack([T.T,np.arange(len(T))]).T
tr=len(_X_tr)
val=len(_X_val)
T[:,2][tr:] -= tr
T[:,2][tr+val:] -= val
_X_tr[0:2],_X_val[0:2],_X_te[0:2],T[0:2],T[tr:tr+2],T[tr+val:tr+val+2]

([['i', '️', 'donnie', 'excellent', 'thematic', 'consistency'],
  ['tanning',
   'for',
   'the',
   'first',
   'time',
   'in',
   'months',
   'i',
   'love',
   'you',
   'sun',
   '️']],
 [['that’s',
   'a',
   'fact',
   'lemme',
   'just',
   'stay',
   'positive',
   'thanks',
   'sis'],
  ['corona', 'or', 'not', 'twitter', 'don’t', 'stop']],
 [['now',
   'is',
   'one',
   'the',
   'most',
   'important',
   'times',
   'to',
   'drink',
   'ratwater',
   '️thats',
   'all',
   'im',
   'going',
   'to',
   'say',
   'for',
   'now'],
  ['this', 'drive', 'always', 'sucks', 'after', 'a', 'while']],
 array([[ 0,  6,  0],
        [ 0, 12,  1]]),
 array([[1, 9, 0],
        [1, 6, 1]]),
 array([[ 2, 18,  0],
        [ 2,  7,  1]]))

In [20]:
i = 0
for t in T:
    for j in range(t[1]):
        if t[0] == 0:
            _X_tr[t[2]][j] = runonsentence[i]
        elif t[0] == 1:
            _X_val[t[2]][j] = runonsentence[i]
        else:
            _X_te[t[2]][j] = runonsentence[i]
        i += 1
_X_tr[0:2],_X_val[0:2],_X_te[0:2]

([['i', '️', '<UNK>', 'excellent', '<UNK>', '<UNK>'],
  ['<UNK>',
   'for',
   'the',
   'first',
   'time',
   'in',
   'months',
   'i',
   'love',
   'you',
   'sun',
   '️']],
 [['that’s',
   'a',
   'fact',
   'lemme',
   'just',
   'stay',
   'positive',
   'thanks',
   'sis'],
  ['corona', 'or', 'not', 'twitter', 'don’t', 'stop']],
 [['now',
   'is',
   'one',
   'the',
   'most',
   'important',
   'times',
   'to',
   'drink',
   '<UNK>',
   '<UNK>',
   'all',
   'im',
   'going',
   'to',
   'say',
   'for',
   'now'],
  ['this', 'drive', 'always', 'sucks', 'after', 'a', 'while']])

In [21]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
import pickle
with open('emoji_sentiment', 'rb') as fp:
    emoji_sentiment = pickle.load(fp)

def convert_to_id(_X):
    return [[word2id[word] for word in tweet] for tweet in _X]

def convert_to_one_hot(_y):
    #return enc.transform(np.array(_y).reshape(-1,1)).toarray()
    _y = np.array(_y) - np.array(_y).min()
    _y = _y * (len(np.unique(_y))-1) / max(_y)
    print(_y)
    return to_categorical(_y, num_classes=len(np.unique(list(emoji_sentiment.values()))))

word2id = {}
for i,word in enumerate(word_counts):
    word2id[word] = i

max_words = max([len(x) for x in _X])
X_tr= sequence.pad_sequences(convert_to_id(_X_tr), maxlen=max_words)
X_val = sequence.pad_sequences(convert_to_id(_X_val), maxlen=max_words)
X_te = sequence.pad_sequences(convert_to_id(_X_te), maxlen=max_words)

y_tr = convert_to_one_hot(_y_tr)
y_val = convert_to_one_hot(_y_val)
y_te = convert_to_one_hot(_y_te)

X_tr[:20], y_tr[:20]

[2. 2. 1. ... 0. 2. 2.]
[2. 2. 2. ... 1. 0. 2.]
[1. 2. 1. ... 0. 1. 2.]


(array([[   0,    0,    0, ..., 1494,  101,  101],
        [   0,    0,    0, ..., 4903, 4156, 5037],
        [   0,    0,    0, ..., 1822, 3949, 1559],
        ...,
        [   0,    0,    0, ..., 2117,  230, 4396],
        [   0,    0,    0, ...,    0,    0,  101],
        [   0,    0,    0, ..., 1397, 5015, 5037]], dtype=int32),
 array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.]], dtype=float32))

This concludes the data grooming. Run the analysis!

In [49]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.arange(len(y_tr[0])),
                                                  y=np.argmax(y_tr,axis=1))
# class_weights = class_weight.compute_class_weight('balanced',
#                                                   classes=np.arange(len(y_tr[0]))-1,
#                                                   y=_y_tr)

print(np.sum(y_tr,axis=0)*class_weights)

class_weights = {i:w for i,w in zip(range(len(y_tr[0])),class_weights)}

class_weights

[110613.33333333 110613.33333333 110613.33333333]


{0: 1.1353574336761576, 1: 1.0865002733931197, 2: 0.8341439994369327}

In [141]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model=Sequential()

embedding_size=32
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))

n_neurons = ceil(np.log2(len(X_tr)))
# model.add(Bidirectional(LSTM(n_neurons, return_sequences=True),
#                         backward_layer=LSTM(n_neurons, return_sequences=True,
#                                             activation='relu', go_backwards=True)))
# model.add(Dropout(0.2))
# model.add(LSTM(n_neurons, return_sequences=True))
# model.add(Dropout(0.2))
model.add(LSTM(n_neurons))
model.add(Dropout(0.2))
if len(class_weights) == 2:
    model.add(Dense(2, activation='sigmoid'))
else:
    model.add(Dense(len(class_weights), activation='softmax'))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 142, 32)           161344    
_________________________________________________________________
lstm_12 (LSTM)               (None, 19)                3952      
_________________________________________________________________
dropout_11 (Dropout)         (None, 19)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 60        
Total params: 165,356
Trainable params: 165,356
Non-trainable params: 0
_________________________________________________________________


In [142]:
loss = 'binary_crossentropy' if len(class_weights) == 2 else 'categorical_crossentropy'
model.compile(loss=loss, optimizer='Nadam', metrics=['accuracy'])

In [143]:
from tensorflow.keras.callbacks import ModelCheckpoint
filepath = 'temp.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')
num_epochs = 10
model.fit(X_tr, y_tr, validation_data=(X_val, y_val), batch_size=batch_size,
          epochs=num_epochs, class_weight=class_weights, callbacks=[checkpoint])

Epoch 1/10
5185/5185 [==============================] - ETA: 0s - loss: 1.0210 - accuracy: 0.4571
Epoch 00001: val_accuracy improved from -inf to 0.48571, saving model to temp.hdf5
5185/5185 [==============================] - 232s 45ms/step - loss: 1.0210 - accuracy: 0.4571 - val_loss: 0.9947 - val_accuracy: 0.4857
Epoch 2/10
5184/5185 [============================>.] - ETA: 0s - loss: 0.9940 - accuracy: 0.4838
Epoch 00002: val_accuracy did not improve from 0.48571
5185/5185 [==============================] - 234s 45ms/step - loss: 0.9940 - accuracy: 0.4837 - val_loss: 1.0010 - val_accuracy: 0.4742
Epoch 3/10
5185/5185 [==============================] - ETA: 0s - loss: 0.9847 - accuracy: 0.4915
Epoch 00003: val_accuracy did not improve from 0.48571
5185/5185 [==============================] - 228s 44ms/step - loss: 0.9847 - accuracy: 0.4915 - val_loss: 0.9952 - val_accuracy: 0.4800
Epoch 4/10
5185/5185 [==============================] - ETA: 0s - loss: 0.9774 - accuracy: 0.4976 ETA: 1s

In [144]:
model.load_weights(filepath)
yhat_te = model.predict(X_te)

In [145]:
wrong = np.argmax(yhat_te,axis=1) != np.argmax(y_te,axis=1)
d = Counter()
for t in np.sort(np.argmax(y_te,axis=1)[wrong]):
    d[t] += 1
dh = Counter()
for t in np.sort(np.argmax(yhat_te,axis=1)[wrong]):
    dh[t] += 1
d,dh,np.mean(1-wrong)

(Counter({0: 54167, 1: 56219, 2: 61545}),
 Counter({0: 45587, 1: 60885, 2: 65459}),
 0.4818062081624165)

In [147]:
scores = model.evaluate(X_te, y_te, verbose=0)
'Test accuracy:', scores[1]

('Test accuracy:', 0.481806218624115)

In [148]:
covid = np.zeros(len(_X_te), dtype=np.bool)
for i,x in enumerate(_X_te):
    hascovid = False
    for w in x:
        if ('covid' in w) or (w == 'coronavirus') or (w == 'corona') or (w == 'rona'):
            hascovid = True
    if hascovid:
        covid[i] = True
yhat_te[covid * wrong],y_te[covid * wrong],1-sum(covid * wrong)/sum(covid)

(array([[0.34851447, 0.35902765, 0.29245788],
        [0.20830192, 0.47995296, 0.3117452 ],
        [0.2982721 , 0.42380086, 0.277927  ],
        ...,
        [0.43346646, 0.32144463, 0.24508892],
        [0.53167665, 0.2701659 , 0.19815741],
        [0.2938578 , 0.35160023, 0.354542  ]], dtype=float32),
 array([[1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        ...,
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=float32),
 0.47996065896238016)